In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

In [2]:
df = pd.read_csv("./dataset/BMDS_data.csv")
df

,materials,bilayer_materials,ground_state_Stacking,a_mono,a,b,DFT_energy_bi,magnetic_moment,formation_energy,area,d,space_group,no_soc_bandgap,soc_bandgap,dielectric_constant,soc_energy,del_e_bilayer_bottom_and_mono
0,BiMoAs_814,BMDB_BiMoAs-BiMoAs_1,AA,3.321954,3.336951,3.336951,-39.623672,0.0,-0.297236,9.643400,2.701272,P3m1,0.0000,0.0000,133.587840,-0.612305,0.112123
1,BiTaAs_815,BMDB_BiTaAs-BiTaAs_2,AA,3.493583,3.438187,3.438187,-42.397422,0.0,-0.333881,10.237285,2.680038,P3m1,0.0000,0.0000,34.725118,-1.040611,0.110058
2,AsClTe_820,BMDB_ClAsTe-ClAsTe_3,AA,3.934365,3.924260,3.924260,-22.106501,0.0,-0.156248,13.336804,3.232067,P3m1,0.4441,0.2889,7.557986,-0.078199,-0.028724
3,NbAsSb_825,BMDB_SbNbAs-SbNbAs_4,AA,3.401773,3.361659,3.361659,-40.392551,0.0,-0.332292,9.786863,2.435314,P3m1,0.0000,0.0000,60.535916,-0.115801,0.105072
4,BiClS_834,BMDB_ClBiS-ClBiS_5,AB1,4.066563,4.052372,4.052372,-24.352339,0.0,-0.135522,14.179504,2.913275,P3m1,1.6105,1.0477,4.177573,-0.403225,-0.007675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,C2_3176,BMDB_C2-C2_756,AB,2.468276,2.467186,2.467186,-37.124373,0.0,-0.059683,5.271681,3.493450,P-3m1,0.0000,0.0762,109.014674,-0.000015,-0.000421
756,Ge2_132,BMDB_Ge2-Ge2_757,AA,4.042847,4.024250,4.024250,-17.024606,0.0,-0.217768,14.024847,1.870524,P-3m1,0.0000,0.0000,9.745150,-0.018109,-0.003826
757,P2_1012,BMDB_P2-P2_758,AA,3.276642,3.269251,3.269251,-21.859284,0.0,-0.102141,9.256140,3.420515,P-3m1,1.2669,1.2667,3.748797,-0.000827,-0.001281
758,Sb2_2159,BMDB_Sb2-Sb2_759,AA,4.119721,4.134183,4.134183,-16.850371,0.0,-0.160182,14.801549,3.177799,P-3m1,0.5061,0.4302,8.097981,-0.133777,0.000316


In [3]:
from sklearn.model_selection import train_test_split

df = df.drop(['materials', 'bilayer_materials', 'ground_state_Stacking', 'space_group'], axis=1)

In [4]:
from hyperimpute.plugins.imputers import Imputers, ImputerPlugin
from Impute import fill_with_et

imputers = Imputers()


class EtImputer(ImputerPlugin):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._model = fill_with_et

    @staticmethod
    def name():
        return "et"

    @staticmethod
    def hyperparameter_space():
        return []

    def _fit(self, *args, **kwargs):
        return self

    def _transform(self, df):
        # 按照缺失值的比例进行排序
        miss_rate = df.isnull().sum() / df.shape[0]
        cols = miss_rate.sort_values().index.tolist()
        cols = [col for col in cols if miss_rate[col] > 0]
        for col in cols:
            df_col_filled = self._model(df, col)
            df[col] = df_col_filled[col]
        return df


imputers.add("et", EtImputer)

hyper = imputers.get("hyperimpute", n_inner_iter=1)
et = imputers.get("et")
missforest = imputers.get("missforest", max_iter=1)
gain = imputers.get("gain", n_epochs=10)
sinkhorn = imputers.get("sinkhorn", n_epochs=10)
mean = imputers.get("mean")

d:\software\miniconda\Lib\site-packages\pydantic\_internal\_fields.py:149: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [5]:
target_col = "soc_bandgap"

In [6]:
# from sklearn.model_selection import StratifiedKFold, KFold
# from sklearn.ensemble import RandomForestRegressor
# from hyperimpute.plugins.utils.simulate import simulate_nan
# from sklearn.metrics import r2_score, mean_absolute_error as MAE
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import scienceplots
# import warnings

# warnings.filterwarnings("ignore")

# plt.style.use(['science','no-latex'])
# # set font as times new roman
# plt.rcParams["font.family"] = "Times New Roman"
# plt.figure(figsize=(12, 8))

# X = df.drop([target_col], axis=1)
# y = df[target_col]

# kf = KFold(n_splits=10, shuffle=True, random_state=42)

# np.random.seed(25)

# scores_et = []
# r2_scores_et = []

# scores_hyper = []
# r2_scores_hyper = []

# scores_missforest = []
# r2_scores_missforest = []

# scores_ori = []
# r2_scores_ori = []

# scores_gain = []
# r2_scores_gain = []

# scores_sinkhorn = []
# r2_scores_sinkhorn = []

# scores_mean = []
# r2_scores_mean = []

# for i, (train_idx, test_idx) in enumerate(kf.split(X, y)):
#     X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
#     y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
#     df_train = pd.concat([X_train, y_train], axis=1)
#     df_test = pd.concat([X_test, y_test], axis=1)
#     # reset the index
#     df_train = df_train.reset_index(drop=True)
#     df_test = df_test.reset_index(drop=True)
#     X_train = df_train.drop([target_col], axis=1)
#     y_train = df_train[target_col]
#     X_test = df_test.drop([target_col], axis=1)
#     y_test = df_test[target_col]
#     cols = X_train.columns
#     X_train = X_train.to_numpy()
#     ret = simulate_nan(X_train, 0.1)
#     X_train_null = ret['X_incomp']
#     mask = ret['mask']
#     X_train_null = pd.DataFrame(X_train_null, columns=cols)

#     X_imputed_hyper = hyper.fit_transform(X_train_null.copy())
#     X_imputed_missforest = missforest.fit_transform(X_train_null.copy())
#     X_imputed_et = et.fit_transform(X_train_null.copy())
#     X_imputed_gain = gain.fit_transform(X_train_null.copy())
#     X_imputed_sinkhorn = sinkhorn.fit_transform(X_train_null.copy())
#     X_imputed_mean = mean.fit_transform(X_train_null.copy())

#     rf = RandomForestRegressor(n_estimators=500)
#     rf.fit(X_imputed_hyper, y_train)
#     y_pred_hyper = rf.predict(X_test)
#     hyper_score = MAE(y_test, y_pred_hyper)
#     r2_hyper = r2_score(y_test, y_pred_hyper)
#     print("Hyperimpute score: ", hyper_score)

#     rf = RandomForestRegressor(n_estimators=500)
#     rf.fit(X_imputed_missforest, y_train)
#     y_pred_missforest = rf.predict(X_test)
#     missforest_score = MAE(y_test, y_pred_missforest)
#     r2_missforest = r2_score(y_test, y_pred_missforest)
#     print("Missforest score: ", missforest_score)

#     rf = RandomForestRegressor(n_estimators=500)
#     rf.fit(X_imputed_et, y_train)
#     y_pred_et = rf.predict(X_test)
#     et_score = MAE(y_test, y_pred_et)
#     r2_et = r2_score(y_test, y_pred_et)
#     print("ET score: ", et_score)

#     rf = RandomForestRegressor(n_estimators=500)
#     rf.fit(X_train, y_train)
#     y_pred = rf.predict(X_test)
#     ori_score = MAE(y_test, y_pred)
#     r2_ori = r2_score(y_test, y_pred)
#     print("Original score: ", ori_score)

#     rf = RandomForestRegressor(n_estimators=500)
#     rf.fit(X_imputed_gain, y_train)
#     y_pred_gain = rf.predict(X_test)
#     gain_score = MAE(y_test, y_pred_gain)
#     r2_gain = r2_score(y_test, y_pred_gain)
#     print("Gain score: ", gain_score)

#     rf = RandomForestRegressor(n_estimators=500)
#     rf.fit(X_imputed_sinkhorn, y_train)
#     y_pred_sinkhorn = rf.predict(X_test)
#     sinkhorn_score = MAE(y_test, y_pred_sinkhorn)
#     r2_sinkhorn = r2_score(y_test, y_pred_sinkhorn)
#     print("Sinkhorn score: ", sinkhorn_score)

#     rf = RandomForestRegressor(n_estimators=500)
#     rf.fit(X_imputed_mean, y_train)
#     y_pred_mean = rf.predict(X_test)
#     mean_score = MAE(y_test, y_pred_mean)
#     r2_mean = r2_score(y_test, y_pred_mean)
#     print("Mean score: ", mean_score)

#     scores_et.append(et_score)
#     scores_hyper.append(hyper_score)
#     scores_missforest.append(missforest_score)
#     scores_gain.append(gain_score)
#     scores_sinkhorn.append(sinkhorn_score)
#     scores_mean.append(mean_score)
#     scores_ori.append(ori_score)

#     r2_scores_et.append(r2_et)
#     r2_scores_hyper.append(r2_hyper)
#     r2_scores_missforest.append(r2_missforest)
#     r2_scores_gain.append(r2_gain)
#     r2_scores_sinkhorn.append(r2_sinkhorn)
#     r2_scores_mean.append(r2_mean)
#     r2_scores_ori.append(r2_ori)

#     # plot
#     sns.scatterplot(x=y_test, y=y_pred, s=120, marker='o', linewidths=2, facecolors='#7ED9D9',
#                 edgecolors='#7ED9D9')

#     sns.scatterplot(x=y_test, y=y_pred_hyper,  marker='v', s=150, linewidths=2,
#                 facecolors='none', edgecolors='#9467BD')
#     sns.scatterplot(x=y_test, y=y_pred_missforest, marker='^', s=150,linewidths=2,
#                 facecolors='none', edgecolors='#B3DE69')
#     sns.scatterplot(x=y_test, y=y_pred_gain, marker='>', s=150,linewidths=2,
#                 facecolors='none', edgecolors='#FFC0D9')
#     sns.scatterplot(x=y_test, y=y_pred_sinkhorn,  marker='<', s=150,linewidths=2,
#                  facecolors='none', edgecolors='#5FBDFF')
#     sns.scatterplot(x=y_test, y=y_pred_mean, marker='s', s=150,linewidths=2,
#                 facecolors='none',edgecolors='#FDBF6F')
#     sns.scatterplot(x=y_test, y=y_pred_et, marker='*', s=300, linewidths=0,
#                 color='#F35F5F', edgecolors='#F35F5F')
#     print("=====================================")

# plt.legend(["Original", "HyperImpute", "MissForest", "Gain", "Sinkhorn", "Mean", "MatImpute"], loc='upper left',
#            fontsize=20, frameon=False, bbox_to_anchor=(0, 0.95))

# # plt.legend(['Original: MAE={:.2f}, $R^2$={:.2f}'.format(np.mean(scores_ori), np.mean(r2_scores_ori)),
# #             'MatImpute: MAE={:.2f}, $R^2$={:.2f}'.format(np.mean(scores_et), np.mean(r2_scores_et)),
# #             'Hyperimpute: MAE={:.2f}, $R^2$={:.2f}'.format(np.mean(scores_hyper), np.mean(r2_scores_hyper)),
# #             'Missforest: MAE={:.2f}, $R^2$={:.2f}'.format(np.mean(scores_missforest), np.mean(r2_scores_missforest)),
# #             'Gain: MAE={:.2f}, $R^2$={:.2f}'.format(np.mean(scores_gain), np.mean(r2_scores_gain)),
# #             'Sinkhorn: MAE={:.2f}, $R^2$={:.2f}'.format(np.mean(scores_sinkhorn), np.mean(r2_scores_sinkhorn)),
# #             'Mean: MAE={:.2f}, $R^2$={:.2f}'.format(np.mean(scores_mean), np.mean(r2_scores_mean))], loc='upper left',
# #            fontsize=16,bbox_to_anchor=(0.0, 0.95))

# plt.plot([np.min(y), np.max(y)], [np.min(y), np.max(y)], 'k--', lw=2, color='#161A30')
# plt.xlabel('True Values', fontsize=24)
# plt.ylabel('Predict Values', fontsize=24)
# plt.xticks(fontsize=24)
# plt.yticks(fontsize=24)
# plt.tight_layout()
# plt.savefig('results/pipeline-regression-BMDS.png', dpi=300)

In [7]:
# print("Hyperimpute score:{:.2f}±{:.2f}".format(np.mean(scores_hyper), np.std(scores_hyper)))
# print("Missforest score:{:.2f}±{:.2f}".format(np.mean(scores_missforest), np.std(scores_missforest)))
# print("ET score:{:.2f}±{:.2f}".format(np.mean(scores_et), np.std(scores_et)))
# print("Original score:{:.2f}±{:.2f}".format(np.mean(scores_ori), np.std(scores_ori)))
# print("Gain score:{:.2f}±{:.2f}".format(np.mean(scores_gain), np.std(scores_gain)))
# print("Sinkhorn score:{:.2f}±{:.2f}".format(np.mean(scores_sinkhorn), np.std(scores_sinkhorn)))
# print("Mean score:{:.2f}±{:.2f}".format(np.mean(scores_mean), np.std(scores_mean)))

In [8]:
# print("Hyperimpute r2:{:.2f}±{:.2f}".format(np.mean(r2_scores_hyper), np.std(r2_scores_hyper)))
# print("Missforest r2:{:.2f}±{:.2f}".format(np.mean(r2_scores_missforest), np.std(r2_scores_missforest)))
# print("ET r2:{:.2f}±{:.2f}".format(np.mean(r2_scores_et), np.std(r2_scores_et)))
# print("Original r2:{:.2f}±{:.2f}".format(np.mean(r2_scores_ori), np.std(r2_scores_ori)))
# print("Gain r2:{:.2f}±{:.2f}".format(np.mean(r2_scores_gain), np.std(r2_scores_gain)))
# print("Sinkhorn r2:{:.2f}±{:.2f}".format(np.mean(r2_scores_sinkhorn), np.std(r2_scores_sinkhorn)))
# print("Mean r2:{:.2f}±{:.2f}".format(np.mean(r2_scores_mean), np.std(r2_scores_mean)))

In [9]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.ensemble import RandomForestRegressor
from hyperimpute.plugins.utils.simulate import simulate_nan
from sklearn.metrics import r2_score, mean_absolute_error as MAE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots
import warnings
from tqdm import tqdm

warnings.filterwarnings("ignore")
X = df.drop([target_col], axis=1)
y = df[target_col]
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)


original_scores = []
matimpute_scores = []
hyperimpute_scores = []
missforest_scores = []
gain_scores = []
sinkhorn_scores = []
mean_scores = []

for ratio in tqdm([0.1, 0.2, 0.3, 0.4, 0.5]):
    original_score = []
    matimpute_score = []
    hyperimpute_score = []
    missforest_score = []
    gain_score = []
    sinkhorn_score = []
    mean_score = []

    for i, (train_idx, test_idx) in enumerate(kf.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        df_train = pd.concat([X_train, y_train], axis=1)
        df_test = pd.concat([X_test, y_test], axis=1)
        # reset the index
        df_train = df_train.reset_index(drop=True)
        df_test = df_test.reset_index(drop=True)
        X_train = df_train.drop([target_col], axis=1)
        y_train = df_train[target_col]
        X_test = df_test.drop([target_col], axis=1)
        y_test = df_test[target_col]


        cols = X_train.columns
        X_train = X_train.to_numpy()
        ret = simulate_nan(X_train, ratio)
        X_train_null = ret['X_incomp']
        mask = ret['mask']
        X_train_null = pd.DataFrame(X_train_null, columns=cols)

        X_imputed_hyper = hyper.fit_transform(X_train_null.copy())
        X_imputed_missforest = missforest.fit_transform(X_train_null.copy())
        X_imputed_et = et.fit_transform(X_train_null.copy())
        X_imputed_gain = gain.fit_transform(X_train_null.copy())
        X_imputed_sinkhorn = sinkhorn.fit_transform(X_train_null.copy())
        X_imputed_mean = mean.fit_transform(X_train_null.copy())
        
        rf = RandomForestRegressor(n_estimators=500)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)
        original_score.append(r2_score(y_test, y_pred))

        rf = RandomForestRegressor(n_estimators=500)
        rf.fit(X_imputed_et, y_train)
        y_pred_et = rf.predict(X_test)
        matimpute_score.append(r2_score(y_test, y_pred_et))

        rf = RandomForestRegressor(n_estimators=500)
        rf.fit(X_imputed_hyper, y_train)
        y_pred_hyper = rf.predict(X_test)
        hyperimpute_score.append(r2_score(y_test, y_pred_hyper))

        rf = RandomForestRegressor(n_estimators=500)
        rf.fit(X_imputed_missforest, y_train)
        y_pred_missforest = rf.predict(X_test)
        missforest_score.append(r2_score(y_test, y_pred_missforest))

        rf = RandomForestRegressor(n_estimators=500)
        rf.fit(X_imputed_gain, y_train)
        y_pred_gain = rf.predict(X_test)
        gain_score.append(r2_score(y_test, y_pred_gain))

        rf = RandomForestRegressor(n_estimators=500)
        rf.fit(X_imputed_sinkhorn, y_train)
        y_pred_sinkhorn = rf.predict(X_test)
        sinkhorn_score.append(r2_score(y_test, y_pred_sinkhorn))

        rf = RandomForestRegressor(n_estimators=500)
        rf.fit(X_imputed_mean, y_train)
        y_pred_mean = rf.predict(X_test)
        mean_score.append(r2_score(y_test, y_pred_mean))
    original_scores.append(original_score)
    matimpute_scores.append(matimpute_score)
    hyperimpute_scores.append(hyperimpute_score)
    missforest_scores.append(missforest_score)
    gain_scores.append(gain_score)
    sinkhorn_scores.append(sinkhorn_score)
    mean_scores.append(mean_score)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [27:31<00:00, 330.35s/it]


In [10]:
# save the results

results = [
    original_scores,
    matimpute_scores,
    hyperimpute_scores,
    missforest_scores,
    gain_scores,
    sinkhorn_scores,
    mean_scores
]

# save to npy file
np.save("results/result_bmds.npy", results)

In [12]:
results[0]

[[0.980089538183737,
  0.9751191934374196,
  0.9934397556113388,
  0.9065630576482222,
  0.9851205122930058,
  0.9876775586988623,
  0.9919687660588724,
  0.9805390186039527,
  0.9855702017990952,
  0.9769572219289949],
 [0.980089538183737,
  0.9751191934374196,
  0.9934397556113388,
  0.9065630576482222,
  0.9851205122930058,
  0.9876775586988623,
  0.9919687660588724,
  0.9805390186039527,
  0.9855702017990952,
  0.9769572219289949],
 [0.9784975333052699,
  0.9734256062671636,
  0.9928994294143509,
  0.906067263911911,
  0.9843416421584313,
  0.9893125540790303,
  0.9924442954230941,
  0.9808835613583954,
  0.9855513280581069,
  0.9773687835923855],
 [0.979078666292869,
  0.9734957162427021,
  0.9933239709946504,
  0.9066300095500657,
  0.9840816383369803,
  0.9880709214738861,
  0.9913304096470106,
  0.9801844608414929,
  0.984327320688767,
  0.9772246461907435],
 [0.9778202177642205,
  0.9738999027318419,
  0.9931738968309712,
  0.9070064124498309,
  0.9845476736447989,
  0.9855146

In [11]:
# # plot the results
# import scienceplots

# plt.style.use(['nature'])
# # set font as times new roman
# plt.rcParams["font.family"] = "Times New Roman"
# plt.figure(figsize=(8, 6))

# # set no grid
# plt.grid(False)

# sns.lineplot(x=[0.1, 0.2, 0.3, 0.4, 0.5], y=original_scores, label='Original', marker='o',color='#7ED9D9',lw=2, markersize=8)
# sns.lineplot(x=[0.1, 0.2, 0.3, 0.4, 0.5], y=matimpute_scores, label='MatImpute', marker='*',color='#F35F5F',lw=2, markersize=12)
# sns.lineplot(x=[0.1, 0.2, 0.3, 0.4, 0.5], y=hyperimpute_scores, label='HyperImpute', marker='v',color='#9467BD',lw=2, markersize=8)
# sns.lineplot(x=[0.1, 0.2, 0.3, 0.4, 0.5], y=missforest_scores, label='MissForest', marker='^',color='#B3DE69',lw=2, markersize=8)
# sns.lineplot(x=[0.1, 0.2, 0.3, 0.4, 0.5], y=gain_scores, label='Gain', marker='>',color='#FFC0D9',lw=2, markersize=8)
# sns.lineplot(x=[0.1, 0.2, 0.3, 0.4, 0.5], y=sinkhorn_scores, label='Sinkhorn', marker='<',color='#5FBDFF',lw=2, markersize=8)
# sns.lineplot(x=[0.1, 0.2, 0.3, 0.4, 0.5], y=mean_scores, label='Mean', marker='s',color='#FDBF6E',lw=2, markersize=7)

# plt.xlabel('Missing Ratio', fontsize=24)
# plt.ylabel('$R^2$ Score',fontsize=24)
# plt.tick_params(labelsize=24)
# plt.legend([])
# plt.tight_layout()
# plt.savefig(f'results/missing-ratio-BMDS.png', dpi=300)